In [28]:
import cv2
import os
import numpy as  np
import matplotlib.pyplot as plt
from scipy.fftpack import dct   ## forward DCT
from scipy.fftpack import idct ## inverse DCT

In [29]:
datadir_face_train = 'trainingData/'
trainingFaceImages = os.listdir(datadir_face_train)

In [30]:
def make_lut_u():
    return np.array([[[i,255-i,0] for i in range(256)]],dtype=np.uint8)

def make_lut_v():
    return np.array([[[0,255-i,i] for i in range(256)]],dtype=np.uint8)

In [31]:
### DCT
# implement 2D DCT
def dct2(a):
    return dct(dct(a.T, norm='ortho').T, norm='ortho')

# implement 2D IDCT
def idct2(a):
    return idct(idct(a.T, norm='ortho').T, norm='ortho')    

In [35]:
### Data preprocess
for iFile in trainingFaceImages:
    print(iFile)
    ### Load the training image
    origIm = cv2.imread(datadir_face_train + iFile)
    
    # read lena RGB image and convert to YUV
    img_yuv = cv2.cvtColor(origIm, cv2.COLOR_BGR2YUV)
    y, u, v = cv2.split(img_yuv)

    lut_u, lut_v = make_lut_u(), make_lut_v()
    
    ## forward dct
    imF = dct2(img_yuv)
    
    # do dp pertubation
    ep = 1e-6
    perturbation = ep * np.random.random_sample((imF.shape))
    imF += perturbation
    ## face recognition
    
    ## inverse dct
    imi = idct2(imF)
    
    
    # Convert back to BGR so we can apply the LUT and stack the images
    y = cv2.cvtColor(y, cv2.COLOR_GRAY2BGR)
    u = cv2.cvtColor(u, cv2.COLOR_GRAY2BGR)
    v = cv2.cvtColor(v, cv2.COLOR_GRAY2BGR)

    u_mapped = cv2.LUT(u, lut_u)
    v_mapped = cv2.LUT(v, lut_v)

    result = np.vstack([imi, y, u_mapped, v_mapped])

cv2.imwrite('shed_combo.png', result)

36118999_1983-08-12_2011.jpg
2316999_1959-05-16_2012.jpg
23151499_1988-09-13_2015.jpg
17196499_1976-03-05_2011.jpg
171099_1983-06-10_2012.jpg
32147299_1986-07-28_2013.jpg
880099_1948-07-16_2014.jpg


error: OpenCV(4.5.3) /private/var/folders/24/8k48jl6d249_n_qfxwsl6xvm0000gn/T/pip-req-build-xxsyexfp/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
### Training face process 
U_face = np.zeros((nDim, 1)) # mean
sigma_face = np.zeros((nDim, nDim)) # variance of face
diag_sigma_face = np.zeros((nDim, nDim)) # covariance of face
x_training_face = [] # x array

for iFile in trainingFaceImages:
    ### Load the training image
    origIm = cv2.imread(datadir_face_train + iFile)
      
    ### each image x codes:
    # sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)
    # sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)
    # imgeGradient = sobelx + sobely
    # x_training_face.append(np.reshape(imgeGradient, (-1,1)))
    x_training_face.append(np.reshape(origIm, (-1,1)))

xNp_training_face = np.array(x_training_face) 
### MLE mean
U_face = np.mean(xNp_training_face, axis = 0)

#print(f" U_face[100] = {U_face[100]}")

### MLE variance
for x in xNp_training_face:
    sigma_face = sigma_face + (x - U_face) * np.transpose(x-U_face)
sigma_face = sigma_face / len(x_training_face)    

#print(f" diag_sigma[100] = {np.diag(sigma_face)[100]}")

### MLE covaricance
diag_sigma_face = np.diag(np.diag(sigma_face))

#print(f"xNp.shape = {xNp_training_face.shape}")
#print(len(x_training_face))
#print(U_face.shape)
#print(sigma_face.shape)
#print(diag_sigma_face.shape)